# Winepi
A partial implementation of Winepi algorithm described by Mannila, Toivonen and Verkamo in *Discovery of Frequent Episodes in Event Sequences*, 1997.

In [1]:
from episode_miner import Event, EventSequence, collection_of_frequent_episodes
from pprint import pprint

In [2]:
sequence_of_events = (Event('a', 1), Event('b', 2), Event('a', 3), Event('a', 5), Event('b', 8))
event_sequence = EventSequence(sequence_of_events=sequence_of_events, start=0, end=9)

frequent_episodes = collection_of_frequent_episodes(event_sequence, 5, 0.2)
pprint(frequent_episodes)

[9 ('a',),
 10 ('b',),
 5 ('a', 'a'),
 6 ('a', 'b'),
 4 ('b', 'a'),
 3 ('a', 'b', 'a')]


## Only full windows vs all windows

In [3]:
sequence_of_events = (Event('a', 0), Event('b', 4), Event('c', 7))
event_sequence = EventSequence(sequence_of_events=sequence_of_events, start=0, end=8)

frequent_episodes = collection_of_frequent_episodes(event_sequence, 5, 0.2, only_full_windows=True)
pprint(frequent_episodes)
frequent_episodes = collection_of_frequent_episodes(event_sequence, 5, 0.2, only_full_windows=False)
pprint(frequent_episodes)

[1 ('a',), 4 ('b',), 1 ('c',), 1 ('a', 'b'), 1 ('b', 'c')]
[5 ('a',), 5 ('b',), 5 ('c',)]


## Gaps skipping vs no gap skipping
Doesn't work yet.

In [4]:
sequence_of_events = (Event('a', 0), Event('b', 2))
event_sequence = EventSequence(sequence_of_events=sequence_of_events, start=0, end=6)

frequent_episodes = collection_of_frequent_episodes(event_sequence, 4, 0.2, gaps_skipping=True)
pprint(frequent_episodes)
frequent_episodes = collection_of_frequent_episodes(event_sequence, 4, 0.2, gaps_skipping=False)
pprint(frequent_episodes)

[4 ('a',), 4 ('b',), 2 ('a', 'b')]
[4 ('a',), 4 ('b',), 2 ('a', 'b')]
